#  Rent or Buy using Price-to-Rent (P/R) Ratio

## 1. Introduction

"Should I buy or should I rent?" is a common question one ask when moving into their first home or downsizing? Home ownership can represent personal success and stability for some or represent an expensive anchor that reduce travel and relocaion for others. Since 2012, home values have recovered from the 2007 Great Recession that originiated from the subprime mortgage crash. Today, it is difficult to determine whether homes values are relative bargains or overpriced money traps. One popular metric to consider in the decision-making process is to reference the Price-to-Rent (P/R) ratio.

We use the following equation to calculate the Price-to-Rent (P/R) ratio:<br>

<i>P/R ratio = median home sales ÷ median rent</i><br>

The results from P/R ratio helps us determine the following:
* High P/R value 21 or greater, rent is better than buy<br>
* Mid P/R value 15 to 20, market value is reasonable<br>
* Low P/R value 14 or less, buy is better than rent<br>

So where in the United States (US) is it better to rent than to buy in today's housing market? 

## 2. Hypothesis 

1. It is better to rent than to buy at 60% or greater locations in today's housing market
2. It is better to buy than to sell at 40% or greater locations in today's housing market


## 3. Data
The following datasets were uploaded from source into GitHub repository to determine P/R ratio, location by zip code, and test hypothesis:

1. Redfin | [2017-2019 Median Home Sales](https://github.com/leonchengmba/Thinkful_capstone1/blob/master/Cap1_data/home_sales.csv)
2. HUD.gov | [2017-2019 Median Fair Market Rental](https://github.com/leonchengmba/Thinkful_capstone1/blob/master/Cap1_data/market_rent.csv)


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [29]:
try:
    raw_home = pd.read_csv('home_sales1.x')
except FileNotFoundError as e:
    print(e)
    print('Attempting to read datasets from online csv files on Github')
    raw_home = pd.read_xlsx('https://github.com/leonchengmba/Thinkful_capstone1/blob/master/Cap1_data/home_sales1.xlsx')

[Errno 2] File b'home_sales1.csv' does not exist: b'home_sales1.csv'
Attempting to read datasets from online csv files on Github


ParserError: Error tokenizing data. C error: Expected 1 fields in line 49, saw 2


In [20]:
print(raw_market.info())
raw_market.head()

NameError: name 'raw_market' is not defined